In [28]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [29]:
la=pd.read_csv('LA.csv')
la.head()

,DISTRICT,NEIGHBOURHOOD,LATITUDE,LONGITUDE
0,FIRST,BOYLE HEIGHTS,34.033888,-118.204444
1,FIRST,CHINATOWN,34.062880,-118.237890
2,FIRST,CIVIC CENTRE,34.054130,-118.244600
3,FIRST,DOWNTOWN L.A,34.057000,-118.238000
4,FIRST,EAGLE ROCK,34.138800,-118.213050


In [30]:
CLIENT_ID = 'DSCNHUN2JCBTJYW4TANBW4SLMQSBABVOQF4FLGJNOQY1R2TD' # your Foursquare ID
CLIENT_SECRET = 'ZGE0WJI1NLXY1RL3ZYKGR1TYXDD4HHCEQX3IOKOAWBYE0NBA' # your Foursquare Secret
VERSION = '20200505' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DSCNHUN2JCBTJYW4TANBW4SLMQSBABVOQF4FLGJNOQY1R2TD
CLIENT_SECRET:ZGE0WJI1NLXY1RL3ZYKGR1TYXDD4HHCEQX3IOKOAWBYE0NBA


In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
pvd_venues=getNearbyVenues(la['NEIGHBOURHOOD'], la['LATITUDE'], la['LONGITUDE'])

In [33]:
pvd_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,BOYLE HEIGHTS,34.033888,-118.204444,Mariscos El Jato,34.037184,-118.204162,Seafood Restaurant
1,BOYLE HEIGHTS,34.033888,-118.204444,La Mascota Bakery,34.032518,-118.211395,Bakery
2,BOYLE HEIGHTS,34.033888,-118.204444,Yum Yum Donuts,34.027976,-118.201088,Donut Shop
3,BOYLE HEIGHTS,34.033888,-118.204444,Ramirez Liquor & Kegs Delivery,34.034776,-118.215187,Liquor Store
4,BOYLE HEIGHTS,34.033888,-118.204444,Artic Hot Spot,34.040059,-118.210238,Café


In [34]:
pvd_venues.shape

(3964, 7)

In [35]:
pvd_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ARLETA,25,25,25,25,25,25
ARTESIA,93,93,93,93,93,93
ATWATER VILLAGE,78,78,78,78,78,78
AZUSA,41,41,41,41,41,41
BELL AIR,9,9,9,9,9,9
BELL FLOWER,55,55,55,55,55,55
BEVERLY HILLS,100,100,100,100,100,100
BIG PINES,5,5,5,5,5,5
BOYLE HEIGHTS,42,42,42,42,42,42


In [36]:
pvd_onehot = pd.get_dummies(pvd_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
pvd_onehot['Neighborhood'] = pvd_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [pvd_onehot.columns[-1]] + list(pvd_onehot.columns[:-1])
pvd_onehot = pvd_onehot[fixed_columns]

pvd_onehot.head()

,Yoshoku Restaurant,ATM,Accessories Store,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus School,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Quad,College Rec Center,College Residence Hall,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Luggage Store,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pop-Up Shop,Portuguese Restaurant,Post Office,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping

In [37]:
num_top_venues=10
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [13]:
pvd_grouped = pvd_onehot.groupby('Neighborhood').mean().reset_index()
pvd_grouped.head()

,Neighborhood,Yoshoku Restaurant,ATM,Accessories Store,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus School,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Quad,College Rec Center,College Residence Hall,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Luggage Store,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pop-Up Shop,Portuguese Restaurant,Post Office,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
pvd_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
pvd_neighborhoods_venues_sorted['Neighborhood'] = pvd_grouped['Neighborhood']

for ind in np.arange(pvd_grouped.shape[0]):
    pvd_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(pvd_grouped.iloc[ind, :], num_top_venues)

pvd_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ARLETA,Mexican Restaurant,Fast Food Restaurant,Video Store,Convenience Store,Dessert Shop,Historic Site,Chinese Restaurant,Clothing Store,Coffee Shop,Supermarket
1,ARTESIA,Indian Restaurant,Chinese Restaurant,Korean Restaurant,Bubble Tea Shop,Grocery Store,Pizza Place,Fast Food Restaurant,Sandwich Place,Thai Restaurant,Vietnamese Restaurant
2,ATWATER VILLAGE,Coffee Shop,Gym / Fitness Center,Thrift / Vintage Store,Liquor Store,Dance Studio,Sandwich Place,Pizza Place,Gym,Vietnamese Restaurant,Mediterranean Restaurant
3,AZUSA,Mexican Restaurant,Thrift / Vintage Store,Coffee Shop,Big Box Store,Pizza Place,Liquor Store,Bank,Convenience Store,Baseball Field,Grocery Store
4,BELL AIR,Hotel Bar,Spa,Café,Golf Course,Hotel,Hotel Pool,Restaurant,Baseball Field,Food,Fast Food Restaurant


In [39]:
kclusters = 6

pvd_grouped_clustering = pvd_grouped.drop('Neighborhood', 1)

# run k-means clustering
pvd_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pvd_grouped_clustering)

# check cluster labels generated for each row in the dataframe
pvd_kmeans.labels_[0:10]

array([2, 2, 1, 2, 1, 2, 1, 1, 2, 1], dtype=int32)

In [40]:

pvd_merged = la
pvd_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', pvd_kmeans.labels_)



pvd_merged = pvd_merged.join(pvd_neighborhoods_venues_sorted.set_index('Neighborhood'), on='NEIGHBOURHOOD')


pvd_merged

,DISTRICT,NEIGHBOURHOOD,LATITUDE,LONGITUDE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,FIRST,BOYLE HEIGHTS,34.033888,-118.204444,2.0,Pizza Place,Mexican Restaurant,Restaurant,Fast Food Restaurant,Sandwich Place,Grocery Store,Convenience Store,Donut Shop,Burger Joint,Liquor Store
1,FIRST,CHINATOWN,34.062880,-118.237890,2.0,Chinese Restaurant,Bakery,Coffee Shop,Plaza,Vietnamese Restaurant,Train Station,Art Gallery,Bar,Sandwich Place,Brewery
2,FIRST,CIVIC CENTRE,34.054130,-118.244600,1.0,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Bar,Music Venue,Plaza,Theater,Bookstore,Mexican Restaurant,Ramen Restaurant
3,FIRST,DOWNTOWN L.A,34.057000,-118.238000,1.0,Plaza,Ramen Restaurant,Coffee Shop,Theater,Supermarket,Japanese Restaurant,Shopping Mall,Fried Chicken Joint,Food Truck,Grocery Store
4,FIRST,EAGLE ROCK,34.138800,-118.213050,2.0,Mexican Restaurant,Chinese Restaurant,American Restaurant,Coffee Shop,Pharmacy,Pizza Place,French Restaurant,Dessert Shop,Thai Restaurant,Bubble Tea Shop
5,FIRST,EL SERENO,34.081100,-118.176900,2.0,Mexican Restaurant,Bank,Trail,South American Restaurant,Food,Brazilian Restaurant,Fast Food Restaurant,Shoe Store,Circus School,Seafood Restaurant
6,FIRST,HIGHLAND PARK,34.111000,-118.189100,2.0,Mexican Restaurant,Bar,Pizza Place,Burger Joint,Bakery,Coffee Shop,Café,Food Truck,Taco Place,Record Shop
7,FIRST,LINCOLN HEIGHTS,34.073500,-118.209600,2.0,Mexican Restaurant,Fast Food Restaurant,Pharmacy,Grocery Store,Juice Bar,Sandwich Place,Video Store,Convenience Store,Taco Place,Coffee Shop
8,FIRST,LITTLE TOKYO,34.050500,-118.239400,1.0,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Brewery,Art Gallery,Coffee Shop,Ice Cream Shop,Bar,Bakery,Cocktail Bar
9,FIRST,SILVERLAKE,34.094400,-118.267500,1.0,Trail,Park,Coffee Shop,Bar,Historic Site,Food Truck,Furniture / Home Store,Record Shop,Vietnamese Restaurant,Spanish Restaurant


In [41]:
address = 'Los Angeles City, LA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of LA are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of LA are 34.0536909, -118.2427666.


In [42]:
map_la = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood,district in zip(la['LATITUDE'], la['LONGITUDE'], la['NEIGHBOURHOOD'], la['DISTRICT']):
    label = '{}, {}'.format(district, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_la)  
    
map_la

In [43]:
pvd_merged=pvd_merged.dropna(axis=0, inplace=False)

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pvd_merged['LATITUDE'], pvd_merged['LONGITUDE'], pvd_merged['NEIGHBOURHOOD'], pvd_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [45]:
pvd_merged.loc[pvd_merged['Cluster Labels'] == 0, pvd_merged.columns[[1] + list(range(5, pvd_merged.shape[1]))]]

,NEIGHBOURHOOD,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,SHADOW HILLS,Stables,Business Service,Yoga Studio,Fish & Chips Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio


In [46]:
pvd_merged.loc[pvd_merged['Cluster Labels'] == 1, pvd_merged.columns[[1] + list(range(5, pvd_merged.shape[1]))]]

,NEIGHBOURHOOD,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,CIVIC CENTRE,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Bar,Music Venue,Plaza,Theater,Bookstore,Mexican Restaurant,Ramen Restaurant
3,DOWNTOWN L.A,Plaza,Ramen Restaurant,Coffee Shop,Theater,Supermarket,Japanese Restaurant,Shopping Mall,Fried Chicken Joint,Food Truck,Grocery Store
8,LITTLE TOKYO,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Brewery,Art Gallery,Coffee Shop,Ice Cream Shop,Bar,Bakery,Cocktail Bar
9,SILVERLAKE,Trail,Park,Coffee Shop,Bar,Historic Site,Food Truck,Furniture / Home Store,Record Shop,Vietnamese Restaurant,Spanish Restaurant
14,MAR VISTA,Japanese Restaurant,Coffee Shop,Café,Mexican Restaurant,Pharmacy,American Restaurant,Bakery,Music Store,Seafood Restaurant,Fast Food Restaurant
15,MIRACLE MILE,Coffee Shop,Art Museum,Convenience Store,Food Truck,Burger Joint,Gym,Sandwich Place,Cosmetics Shop,Dessert Shop,Park
16,PALMS,Coffee Shop,Pizza Place,Sushi Restaurant,Bakery,Yoga Studio,Italian Restaurant,Taco Place,Indian Restaurant,Burger Joint,Café
17,UNIVERSITY PARK,Coffee Shop,American Restaurant,Mexican Restaurant,Burger Joint,Grocery Store,Bakery,Food Truck,Italian Restaurant,Cosmetics Shop,Café
21,ATWATER VILLAGE,Coffee Shop,Gym / Fitness Center,Thrift / Vintage Store,Liquor Store,Dance Studio,Sandwich Place,Pizza Place,Gym,Vietnamese Restaurant,Mediterranean Restaurant
22,BELL AIR,Hotel Bar,Spa,Café,Golf Course,Hotel,Hotel Pool,Restaurant,Baseball Field,Food,Fast Food Restaurant


In [47]:
pvd_merged.loc[pvd_merged['Cluster Labels'] == 2, pvd_merged.columns[[1] + list(range(5, pvd_merged.shape[1]))]]

,NEIGHBOURHOOD,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BOYLE HEIGHTS,Pizza Place,Mexican Restaurant,Restaurant,Fast Food Restaurant,Sandwich Place,Grocery Store,Convenience Store,Donut Shop,Burger Joint,Liquor Store
1,CHINATOWN,Chinese Restaurant,Bakery,Coffee Shop,Plaza,Vietnamese Restaurant,Train Station,Art Gallery,Bar,Sandwich Place,Brewery
4,EAGLE ROCK,Mexican Restaurant,Chinese Restaurant,American Restaurant,Coffee Shop,Pharmacy,Pizza Place,French Restaurant,Dessert Shop,Thai Restaurant,Bubble Tea Shop
5,EL SERENO,Mexican Restaurant,Bank,Trail,South American Restaurant,Food,Brazilian Restaurant,Fast Food Restaurant,Shoe Store,Circus School,Seafood Restaurant
6,HIGHLAND PARK,Mexican Restaurant,Bar,Pizza Place,Burger Joint,Bakery,Coffee Shop,Café,Food Truck,Taco Place,Record Shop
7,LINCOLN HEIGHTS,Mexican Restaurant,Fast Food Restaurant,Pharmacy,Grocery Store,Juice Bar,Sandwich Place,Video Store,Convenience Store,Taco Place,Coffee Shop
10,WESTLAKE,Latin American Restaurant,Fast Food Restaurant,Korean Restaurant,Grocery Store,Mexican Restaurant,Hotel,Sandwich Place,Spa,Park,Coffee Shop
11,CRENSHAW DISTRICT,Mexican Restaurant,Pharmacy,Mobile Phone Shop,Fast Food Restaurant,Light Rail Station,Food Truck,Seafood Restaurant,Sandwich Place,Clothing Store,Southern / Soul Food Restaurant
12,EXPOSITION PARK,Fast Food Restaurant,Science Museum,Sandwich Place,Food Court,Coffee Shop,Pizza Place,Museum,History Museum,Movie Theater,Hot Dog Joint
13,HARBOR GATEWAY,Sandwich Place,Fast Food Restaurant,Sushi Restaurant,Pet Store,Pharmacy,Big Box Store,Paper / Office Supplies Store,Greek Restaurant,Gym,Gas Station


In [48]:
pvd_merged.loc[pvd_merged['Cluster Labels'] == 3, pvd_merged.columns[[1] + list(range(5, pvd_merged.shape[1]))]]

,NEIGHBOURHOOD,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,MALIBU,Surf Spot,Beach,Greek Restaurant,Trail,Restaurant,Yoga Studio,Film Studio,Falafel Restaurant,Farm,Farmers Market


In [49]:
pvd_merged.loc[pvd_merged['Cluster Labels'] == 4, pvd_merged.columns[[1] + list(range(5, pvd_merged.shape[1]))]]

,NEIGHBOURHOOD,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,VERNON,Furniture / Home Store,Food,Snack Place,Grocery Store,Farm,Rental Service,Convenience Store,Office,Mexican Restaurant,Film Studio


In [50]:
pvd_merged.loc[pvd_merged['Cluster Labels'] == 5, pvd_merged.columns[[1] + list(range(5, pvd_merged.shape[1]))]]

,NEIGHBOURHOOD,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
60,RANCHO PALOS VERDES,Trail,Park,Nature Preserve,Food,Yoga Studio,Film Studio,Fabric Shop,Falafel Restaurant,Farm,Farmers Market


In [51]:
pvd_merged.groupby('Cluster Labels').count()['NEIGHBOURHOOD'].to_frame()

,NEIGHBOURHOOD
Cluster Labels,
0.0,1
1.0,27
2.0,37
3.0,1
4.0,1
5.0,1
